<a href="https://colab.research.google.com/github/ksolosina/BCIT/blob/master/ml2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np #библиотека для работы с многомерными массивами данных и математическими операциями над ними
import pandas as pd #библиотека для анализа и обработки данных
pd.set_option('display.max.columns', 100)
import matplotlib.pyplot as plt #простое рисование графиков
import seaborn as sns #удобные дефолтные настройки графиков из matpotlib
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [7]:
import io
from google.colab import files
uploaded = files.upload() #загрузка файлов с компа в коллаб (внизу интерфейс)
data = pd.read_csv(io.BytesIO(uploaded['adult.data.csv'])) #библиотека пандас позволяет создать удобный датафрейм (data)
data.head() #выведем вернхие пять записей датафрейма (по умолчанию 5)

Saving adult.data.csv to adult.data.csv


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [8]:
# 1) Сколько мужчин и женщин ("sex" фича) в этом сете?
data["sex"].value_counts()

Male      21790
Female    10771
Name: sex, dtype: int64

In [9]:
# 2) Какой средний возраст ("age" фича) у мужчин?
data[data["sex"] == "Male"]["age"].mean()

39.43354749885268

In [10]:
# 3) Какой процентаж у жителей ("native-country" фича) Италии? 
print("{0:%}".format(data[data["native-country"] == "Italy"]
                     .shape[0] / data.shape[0]))

0.224195%


In [11]:
# 4-5) Каково среднее и стандартное отклонение возраста для тех, кто зарабатывает более 50 тыс. в год ("salary" фича)
# и тех, кто зарабатывает менее 50 тыс. в год?
ages1 = data.loc[data['salary'] == '>50K', 'age']
ages2 = data.loc[data['salary'] == '<=50K', 'age']
print("The average age of the:\nrich: {0} +- {1} years,\npoor: {2} +- {3} years.".format(
    round(ages1.mean()), round(ages1.std(), 1),
    round(ages2.mean()), round(ages2.std(), 1)))

The average age of the:
rich: 44 +- 10.5 years,
poor: 37 +- 14.0 years.


In [12]:
# 6) Правда, что все с зарплатой >50К имеют высшее образование?
if len(data.loc[data['salary'] == '>50K', 'education'].unique()) > 0:
    print("Ложь")
else:
    print("Правда")

Ложь


In [13]:
# 7.1) Отображение статистики возраста для каждой расы ("race" фича) и каждого пола ("sex" фича).
# Используйте groupby() и describe().
data.groupby(["race", "sex"])["age"].describe()

count       mean        std   min   25%   50%  \
race               sex                                                       
Amer-Indian-Eskimo Female    119.0  37.117647  13.114991  17.0  27.0  36.0   
                   Male      192.0  37.208333  12.049563  17.0  28.0  35.0   
Asian-Pac-Islander Female    346.0  35.089595  12.300845  17.0  25.0  33.0   
                   Male      693.0  39.073593  12.883944  18.0  29.0  37.0   
Black              Female   1555.0  37.854019  12.637197  17.0  28.0  37.0   
                   Male     1569.0  37.682600  12.882612  17.0  27.0  36.0   
Other              Female    109.0  31.678899  11.631599  17.0  23.0  29.0   
                   Male      162.0  34.654321  11.355531  17.0  26.0  32.0   
White              Female   8642.0  36.811618  14.329093  17.0  25.0  35.0   
                   Male    19174.0  39.652498  13.436029  17.0  29.0  38.0   

                             75%   max  
race               sex                  
Amer-Indian-Eskimo Female  46.00  80.0  
                   Male    45.00  82.0  
Asian-Pac-Islander Female  43.75  75.0  
                   Male    46.00  90.0  
Black              Female  46.00  90.0  
                   Male    46.00  90.0  
Other              Female  39.00  74.0  
                   Male    42.00  77.0  
White              Female  46.00  90.0  
                   Male    49.00  90.0

In [14]:

# 7.2) Найти максимальный возраст мужчин американо-индийско-эскимосской расы (Amer-Indian-Eskimo race).
data[(data["race"] == "Amer-Indian-Eskimo") & (data["sex"] == "Male")]["age"].max()

82

In [15]:
# 8) Среди кого больше доля тех, кто много зарабатывает (>50 тыс.):
# женатые или одинокие мужчины (особенность семейного положения)?
# Считать женатыми тех, кто имеет семейное положение, начиная с Married
# (Married-civ-spouse, Married-spouse-absent или Married-AF-spouse), остальные считаются холостяками.
data.loc[(data['sex'] == 'Male') &
     (data['marital-status'].isin(['Never-married', 
                                   'Separated', 
                                   'Divorced',
                                   'Widowed'])), 'salary'].value_counts()

<=50K    7552
>50K      697
Name: salary, dtype: int64

In [16]:
data.loc[(data['sex'] == 'Male') & (data['marital-status'].str.startswith('Married')), 'salary'].value_counts()

<=50K    7576
>50K     5965
Name: salary, dtype: int64

In [17]:
# 9) Какое максимальное количество часов человек работает в неделю ("hours-per-week" фича)?
# Сколько людей работает такое количество часов и каков процент тех, кто зарабатывает много среди них?
max_load = data['hours-per-week'].max()
print("Max time - {0} hours./week.".format(max_load))

num_workaholics = data[data['hours-per-week'] == max_load].shape[0]
print("Total number of such hard workers {0}".format(num_workaholics))

rich_share = float(data[(data['hours-per-week'] == max_load) & (data['salary'] == '>50K')].shape[0]) / num_workaholics
print("Percentage of rich (>50K) among them {0}%".format(int(100 * rich_share)))

Max time - 99 hours./week.
Total number of such hard workers 85
Percentage of rich (>50K) among them 29%


In [18]:
# 10.1) Подсчитайте среднее время работы (hours-per-week) для тех,
# кто зарабатывает мало и много (salary) для каждой страны (native-country).
p = pd.crosstab(data["native-country"], data["salary"],
                values=data['hours-per-week'], aggfunc="mean")
p

salary,<=50K,>50K
native-country,,
?,40.164760,45.547945
Cambodia,41.416667,40.000000
Canada,37.914634,45.641026
China,37.381818,38.900000
Columbia,38.684211,50.000000
Cuba,37.985714,42.440000
Dominican-Republic,42.338235,47.000000
Ecuador,38.041667,48.750000
El-Salvador,36.030928,45.000000


In [19]:
# 10.2) Что это будет для Италии?
p.loc["Italy"]

salary
<=50K    39.625
>50K     45.400
Name: Italy, dtype: float64

In [20]:
!python -m pip install --upgrade pip
!pip install pandasql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.1)
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [39]:
store = files.upload() #загрузка файлов с компа в коллаб (внизу интерфейс)
store = pd.read_csv(io.BytesIO(store['googleplaystore.csv'])) #библиотека пандас позволяет создать удобный датафрейм (data)
store.head() #выведем вернхие пять записей датафрейма (по умолчанию 5)

Saving googleplaystore.csv to googleplaystore (3).csv


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [41]:
store.columns
store = store.dropna()
store = store.drop(['Reviews', 'Size', 'Installs', 'Type', 'Content Rating', 'Genres', 'Last Updated',
                    'Current Ver', 'Android Ver'], 1)
store.head()

,App,Category,Rating,Price
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,0
1,Coloring book moana,ART_AND_DESIGN,3.9,0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,0


In [40]:
sur = files.upload() #загрузка файлов с компа в коллаб (внизу интерфейс)
sur = pd.read_csv(io.BytesIO(sur['googleplaystore_user_reviews.csv'])) #библиотека пандас позволяет создать удобный датафрейм (data)
sur.head() #выведем вернхие пять записей датафрейма (по умолчанию 5)

Saving googleplaystore_user_reviews.csv to googleplaystore_user_reviews (3).csv


,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [45]:
#sur = sur.drop("Translated_Review", 1)
sur = sur.dropna()
sur.head()

,App,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,Positive,1.00,0.533333
1,10 Best Foods for You,Positive,0.25,0.288462
3,10 Best Foods for You,Positive,0.40,0.875000
4,10 Best Foods for You,Positive,1.00,0.300000
5,10 Best Foods for You,Positive,1.00,0.300000


In [46]:
store.merge(sur[["App", "Sentiment"]], on="App").head()

,App,Category,Rating,Price,Sentiment
0,Coloring book moana,ART_AND_DESIGN,3.9,0,Negative
1,Coloring book moana,ART_AND_DESIGN,3.9,0,Negative
2,Coloring book moana,ART_AND_DESIGN,3.9,0,Neutral
3,Coloring book moana,ART_AND_DESIGN,3.9,0,Positive
4,Coloring book moana,ART_AND_DESIGN,3.9,0,Negative


In [50]:
%%timeit
store.merge(sur[["App", "Sentiment"]], on="App")

10 loops, best of 3: 23.9 ms per loop


In [51]:
%%timeit
pysqldf("""SELECT st.App, st.Category, st.Rating, sur.Sentiment
           FROM store AS st JOIN sur
           ON st.App = sur.App""").head()

1 loop, best of 3: 548 ms per loop


In [54]:
%%timeit
sur.groupby("App")["Sentiment_Polarity"].mean()

100 loops, best of 3: 4.4 ms per loop


In [55]:
%%timeit
pysqldf("""SELECT App, AVG(Sentiment_Polarity)
           FROM sur
           GROUP BY App
        """)

1 loop, best of 3: 268 ms per loop
